### Imports

In [30]:
import pandas as pd
import altair as alt
from vega_datasets import data
import numpy as np
import json
from altair import datum
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
from shapely.geometry import MultiPolygon

### 1. Filtering EV Data

In [31]:
state_to_abbreviation = {
    'District Of Columbia': "DC",
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
abbreviation_to_state = {value: key for key, value in state_to_abbreviation.items()}

In [32]:
#Loading in Dataframe
df = pd.read_csv("EV_data_with_Counties_latlongchanges_fixed.csv",low_memory=False)

#Removing all rows except EV Chargers
df = df[df['fuel_type_code'] == 'ELEC']

#Removing Rows if the charger is a future build
df = df[(df["status_code"] == 'E') | (df["status_code"] == 'T')] #Only choosing stations that are currently available or temporarily unavailble
objs = df.copy()
df.drop_duplicates(inplace=True)

#Drop Nan Columns (no county, no date)
df.dropna(subset=['county_id', 'open_date'], how='any', inplace=True)

#Creating Year open Column
df['open_date'] = pd.to_datetime(df['open_date'],format = '%m/%d/%y %H:%M')
df["Year Opened"] = df['open_date'].dt.year.astype(int)

#Gets Remaining County Information
state_counties_all = pd.read_csv("Raw_Data/county_state_ids_important.csv") #file containing county id, state, and county name for every county in usa (https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt)
id_dict = {} #dictionary of ids --> state, county name
for index, row in state_counties_all.iterrows():
    id_dict[row['county_id']] = [row['state'], row['county']]
state_counties_all["Sum Stations"] = ["No stations reported."]*len(state_counties_all) #creates background for ma[]

#Delete any rows that slipped with states not matching county assignment (error of data we were given)
def correct_state(row,id_dict):
    row_id = row["OBJECTID"]
    county_id = int(row["county_id"])
    county = row["county"]
    state = row["state"]

    info_lst = id_dict[county_id]
    state_found = info_lst[0]
    county_found = info_lst[1]
    if county == county_found and state_found == state:
        return 0
    elif county == county_found and state_found != state:
        print(f"OBJECTID '{row_id}' had mismatch of lat/long and state name. State '{state}' was different from '{state_found}' pair with county.")
        return 1
    else:
        print(f"OBJECTID {row_id} has an issue that needs further investigating.")
        return 1
df["drop"] = df.apply(correct_state, id_dict=id_dict, axis=1)
df = df[df["drop"] == 0] #dropping columns where the found county dont match the recorded state from database

#Deleting Puerto Rico Stations and the station in the ocean by Maine (54558)
orig_len = len(df)
df = df[(df["OBJECTID"] != "54558") & (df["state"] != 'PR')]
print(f"Rows deleted with PR or 54558 OBJECTID: {orig_len - len(df)}")
objs_after = df["OBJECTID"].to_list()
objs = objs[~objs["OBJECTID"].isin(objs_after)]

# Transforming the Array (df_stations)
df_stations = df[["OBJECTID","Year Opened","state","county_id","county","open_date"]]
df_stations = df_stations.assign(Number_of_Stations=1)
df_stations["county_id"] = df_stations["county_id"].astype(str)
df_stations = df_stations.groupby(by=["Year Opened","state","county","county_id"],as_index=False).sum(numeric_only=True)
df_stations["county_id"] = df_stations["county_id"].str.split('.').str[0]
df_stations["county_id"] = df_stations["county_id"].astype(int)
df_stations["id"] = df_stations["county_id"]

#Transformation for station locations
df_locations = df[["OBJECTID","Year Opened","state","county_id","county","latitude","longitude","street_address","city","ev_dc_fast_num","ev_level1_evse_num","ev_level2_evse_num"]]
df_locations["county_id"] = df_locations["county_id"].astype(int)
cols_added = ['ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num']
df_locations.loc[:,cols_added] = df_locations.loc[:,cols_added].astype(float)
df_locations.loc[:,cols_added] = df_locations.loc[:,cols_added].fillna(0)
df_locations[cols_added] = df_locations[cols_added].astype(int)
level1_power = 1
level2_power = (7 + 19) / 2
level3_power = (50 + 350) / 2
df_locations.loc[:, "Power_Capacity_Level3_kW"] = df_locations['ev_dc_fast_num'].astype(float) * level3_power
df_locations.loc[:, "Power_Capacity_Level1_kW"] = df_locations['ev_level1_evse_num'].astype(float) * level1_power
df_locations.loc[:, "Power_Capacity_Level2_kW"] = df_locations['ev_level2_evse_num'].astype(float) * level2_power
df_locations.loc[:, "Power_Capacity_Level12kW"] = df_locations['ev_level2_evse_num'].astype(float) * level2_power + df_locations['ev_level1_evse_num'].astype(float) * level1_power
df_locations.loc[:, "Power_Capacity_Level13kW"] = df_locations['ev_level1_evse_num'].astype(float) * level1_power + df_locations['ev_dc_fast_num'].astype(float) * level3_power
df_locations.loc[:, "Power_Capacity_Level23kW"] = df_locations['ev_level2_evse_num'].astype(float) * level2_power + df_locations['ev_dc_fast_num'].astype(float) * level3_power
df_locations.loc[:, "Power_Capacity_kW"] = df_locations['ev_dc_fast_num'].astype(float) * level3_power + df_locations['ev_level1_evse_num'].astype(float) * level1_power + df_locations['ev_level2_evse_num'].astype(float) * level2_power
df_locations

OBJECTID '18975' had mismatch of lat/long and state name. State 'WA' was different from 'HI' pair with county.
OBJECTID '40448' had mismatch of lat/long and state name. State 'FL' was different from 'PR' pair with county.
OBJECTID '40605' had mismatch of lat/long and state name. State 'MS' was different from 'LA' pair with county.
OBJECTID '43022' had mismatch of lat/long and state name. State 'DC' was different from 'MN' pair with county.
OBJECTID '46503' had mismatch of lat/long and state name. State 'WV' was different from 'DC' pair with county.
OBJECTID '59406' had mismatch of lat/long and state name. State 'CA' was different from 'NY' pair with county.
OBJECTID '61648' had mismatch of lat/long and state name. State 'OR' was different from 'HI' pair with county.
OBJECTID '61821' had mismatch of lat/long and state name. State 'MD' was different from 'VA' pair with county.
OBJECTID '63470' had mismatch of lat/long and state name. State 'MO' was different from 'MI' pair with county.
O

/var/folders/6l/z6x890s133g488rww9gwhhrc0000gn/T/ipykernel_77997/3663118168.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locations["county_id"] = df_locations["county_id"].astype(int)
/var/folders/6l/z6x890s133g488rww9gwhhrc0000gn/T/ipykernel_77997/3663118168.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locations[cols_added] = df_locations[cols_added].astype(int)
/var/folders/6l/z6x890s133g488rww9gwhhrc0000gn/T/ipykernel_77997/3663118168.py:73: SettingWithCopyWarning: 
A value is tryin

,OBJECTID,Year Opened,state,county_id,county,latitude,longitude,street_address,city,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,Power_Capacity_Level3_kW,Power_Capacity_Level1_kW,Power_Capacity_Level2_kW,Power_Capacity_Level12kW,Power_Capacity_Level13kW,Power_Capacity_Level23kW,Power_Capacity_kW
133,134,1999,CA,6037,Los Angeles County,34.24831915,-118.387971,11797 Truesdale St,Sun Valley,2,0,57,400.0,0.0,741.0,741.0,400.0,1141.0,1141.0
134,135,1995,CA,6037,Los Angeles County,34.040539,-118.271387,1201 S Figueroa St,Los Angeles,0,0,7,0.0,0.0,91.0,91.0,0.0,91.0,91.0
135,136,1999,CA,6037,Los Angeles County,34.059133,-118.248589,111 N Hope St,Los Angeles,12,0,338,2400.0,0.0,4394.0,4394.0,2400.0,6794.0,6794.0
136,137,2018,CA,6037,Los Angeles County,33.759802,-118.096665,6801 E 2nd St,Long Beach,1,0,19,200.0,0.0,247.0,247.0,200.0,447.0,447.0
137,138,1999,CA,6037,Los Angeles County,33.770508,-118.265628,161 N Island Ave,Wilmington,0,0,10,0.0,0.0,130.0,130.0,0.0,130.0,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105159,83481,2024,CO,8089,Otero County,37.987967,-103.541782,101 Raton Ave,La Junta,0,0,4,0.0,0.0,52.0,52.0,0.0,52.0,52.0
105160,83482,2024,NY,36005,Bronx County,40.80733843,-73.930047,101 Lincoln Ave.,Bronx,0,0,50,0.0,0.0,650.0,650.0,0.0,650.0,650.0
105161,83483,2024,FL,12095,Orange County,28.4203991,-81.502968,1500 E Beach Way,Orlando,6,0,0,1200.0,0.0,0.0,0.0,1200.0,1200.0,1200.0
105162,83484,2024,FL,12095,Orange County,28.408445,-81.524999,2420 North Beach Lane/Flats,Orlando,2,0,0,400.0,0.0,0.0,0.0,400.0,400.0,400.0


### 2. Fixing the Counties Data File

In [33]:
# Load TopoJSON
url = 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json' #this is the url to the counties data from altair
gdf = gpd.read_file(url) #for some reason even just doing this drops the null rows

#Clearing Null Rows
gdf_cleaned = gdf[gdf['geometry'].notnull()] #drop here again just in case
duplicate_ids = gdf[gdf.duplicated(subset='id', keep=False)].sort_values(by="id")
duplicate_ids_vals = duplicate_ids["id"].unique()

duplicate_ids #these are islands around michigan and hawaii (see how there are multiple rows with Polygon Representations rather than MultiPolygons!)

,id,geometry
3112,15007,"POLYGON ((-160.20305 21.77717, -160.22817 21.8..."
3106,15007,"POLYGON ((-159.44570 21.86849, -159.60722 21.8..."
3107,15009,"POLYGON ((-157.01216 21.18201, -156.91525 21.1..."
3108,15009,"POLYGON ((-156.96191 20.73241, -157.03729 20.9..."
3109,15009,"POLYGON ((-156.66759 20.50143, -156.67477 20.5..."
3110,15009,"POLYGON ((-156.37686 20.57502, -156.44505 20.6..."
3153,2201,"POLYGON ((-132.26398 55.73763, -132.22450 55.7..."
3139,2201,"POLYGON ((-132.64445 54.75357, -132.81314 54.9..."
3140,2201,"POLYGON ((-131.34871 54.85563, -131.49228 54.9..."
3141,2201,"POLYGON ((-132.75571 54.99206, -132.88134 55.0..."


#### Example Showing how Multiple Rows for Same Id

In [34]:
gdf[gdf["id"].isin(['17031','15007'])]

,id,geometry
716,17031,"POLYGON ((-88.19822 42.15356, -87.76032 42.151..."
3106,15007,"POLYGON ((-159.44570 21.86849, -159.60722 21.8..."
3112,15007,"POLYGON ((-160.20305 21.77717, -160.22817 21.8..."


#### Correcting Counties Dataframe using MultiPolygons

In [35]:
# Filter out rows with empty geometries
gdf_cleaned = gdf[gdf.geometry.is_empty == False]

# Group by 'id' column
grouped = gdf_cleaned.groupby('id')

# Initialize an empty list to store the rows of the new GeoDataFrame
rows = []

# Iterate over each group
for id, group in grouped:

    # Apply buffer(0) to fix potential topology issues
    buffered_geometries = group.geometry.buffer(0)

    # Combine buffered geometries into a single multipolygon
    multipolygon = buffered_geometries.unary_union

    # If the result is not a MultiPolygon, convert it to one
    if not isinstance(multipolygon, MultiPolygon):
        multipolygon = MultiPolygon([multipolygon])

    # Create a dictionary representing a row with the combined geometry
    row = {'id': id, 'geometry': multipolygon}

    # Append the row to the list
    rows.append(row)

# Create a new GeoDataFrame from the list of rows
result_counties = gpd.GeoDataFrame(rows, columns=['id', 'geometry'])
result_counties

,id,geometry
0,10001,"MULTIPOLYGON (((-75.31265 38.95051, -75.38085 ..."
1,10003,"MULTIPOLYGON (((-75.42392 39.80673, -75.61774 ..."
2,10005,"MULTIPOLYGON (((-75.31265 38.95051, -75.18703 ..."
3,1001,"MULTIPOLYGON (((-86.41434 32.70717, -86.41075 ..."
4,1003,"MULTIPOLYGON (((-87.76391 31.29715, -87.61675 ..."
...,...,...
3192,9007,"MULTIPOLYGON (((-72.46634 41.58364, -72.43045 ..."
3193,9009,"MULTIPOLYGON (((-72.75348 41.57880, -72.74631 ..."
3194,9011,"MULTIPOLYGON (((-71.78796 41.64004, -71.78796 ..."
3195,9013,"MULTIPOLYGON (((-72.13613 42.03055, -72.13613 ..."


### 3. Port Data

#### Source: https://www.transportation.gov/rural/ev/toolkit/ev-basics/charging-speeds
* Fast chargers (50-350 kWh), Level 1 (1 kWh), Level 2 (7-19 kWh)

In [36]:
#Load in Port Columns
df_ports = df[["OBJECTID","Year Opened","state","county_id","county","street_address","city","ev_dc_fast_num","ev_level1_evse_num","ev_level2_evse_num"]].copy()
df_ports.loc[:, ['ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num']] = df_ports.loc[:, ['ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num']].fillna(0)
level1_power = 1
level2_power = (7 + 19) / 2
level3_power = (50 + 350) / 2
df_ports.loc[:, "Power_Capacity_Level3_kW"] = df_ports['ev_dc_fast_num'].astype(float) * level3_power
df_ports.loc[:, "Power_Capacity_Level1_kW"] = df_ports['ev_level1_evse_num'].astype(float) * level1_power
df_ports.loc[:, "Power_Capacity_Level2_kW"] = df_ports['ev_level2_evse_num'].astype(float) * level2_power
df_ports.loc[:, "Power_Capacity_kW"] = df_ports['ev_dc_fast_num'].astype(float) * level3_power + df_ports['ev_level1_evse_num'].astype(float) * level1_power + df_ports['ev_level2_evse_num'].astype(float) * level2_power
df_ports.loc[:, "All_Ports"] =  df_ports['ev_dc_fast_num'].astype(float) + df_ports['ev_level1_evse_num'].astype(float) + df_ports['ev_level2_evse_num'].astype(float)

#Drop Rows where Capacity is Zero (no data recorded)
length_before = len(df_ports)
df_ports = df_ports[df_ports['Power_Capacity_kW'] != 0]
print(f"Rows Dropped for no Power Capcity Data: {length_before-len(df_ports)}")

#Group by County Id and Sum for Each Year
df_ports["county_id"] = df_ports["county_id"].astype(str)
df_ports = df_ports.groupby(by=["Year Opened","state","county","county_id"],as_index=False).sum(numeric_only=True)
df_ports["county_id"] = df_ports["county_id"].str.split('.').str[0]
df_ports["county_id"] = df_ports["county_id"].astype(int)
df_ports["id"] = df_ports["county_id"]
cols_added = ['ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num',"Power_Capacity_kW"]
df_ports[cols_added] = df_ports[cols_added].astype(float)
# df_ports[cols_added] = df_ports[cols_added].replace(0, -1)
df_ports

Rows Dropped for no Power Capcity Data: 10


,Year Opened,state,county,county_id,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,Power_Capacity_Level3_kW,Power_Capacity_Level1_kW,Power_Capacity_Level2_kW,Power_Capacity_kW,All_Ports,id
0,1995,CA,Los Angeles County,6037,0.0,0.0,7.0,0.0,0.0,91.0,91.0,7.0,6037
1,1996,CA,Los Angeles County,6037,0.0,0.0,3.0,0.0,0.0,39.0,39.0,3.0,6037
2,1997,CA,Los Angeles County,6037,0.0,0.0,21.0,0.0,0.0,273.0,273.0,21.0,6037
3,1997,CA,Riverside County,6065,0.0,0.0,4.0,0.0,0.0,52.0,52.0,4.0,6065
4,1997,CA,San Diego County,6073,0.0,0.0,1.0,0.0,0.0,13.0,13.0,1.0,6073
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,2024,WV,Pocahontas County,54075,0.0,0.0,3.0,0.0,0.0,39.0,39.0,3.0,54075
10501,2024,WV,Putnam County,54079,1.0,0.0,2.0,200.0,0.0,26.0,226.0,3.0,54079
10502,2024,WV,Wood County,54107,1.0,0.0,1.0,200.0,0.0,13.0,213.0,2.0,54107
10503,2024,WY,Laramie County,56021,6.0,0.0,0.0,1200.0,0.0,0.0,1200.0,6.0,56021


In [37]:
df_ports.sort_values(by="ev_level1_evse_num",ascending= False,inplace=True)
df_ports_all = df_ports.groupby(by=["state","county","county_id"],as_index=False).sum(numeric_only=True)
df_ports_all.sort_values(by="Power_Capacity_kW",ascending= False,inplace=True)
df_ports_all

,state,county,county_id,Year Opened,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,Power_Capacity_Level3_kW,Power_Capacity_Level1_kW,Power_Capacity_Level2_kW,Power_Capacity_kW,All_Ports,id
135,CA,Los Angeles County,6037,42242,2250.0,144.0,12055.0,450000.0,144.0,156715.0,606859.0,14449.0,126777
146,CA,Orange County,6059,30247,1002.0,1.0,3720.0,200400.0,1.0,48360.0,248761.0,4723.0,90885
159,CA,Santa Clara County,6085,28245,757.0,77.0,4973.0,151400.0,77.0,64649.0,216126.0,5807.0,85190
153,CA,San Diego County,6073,32248,801.0,17.0,3575.0,160200.0,17.0,46475.0,206692.0,4393.0,97168
152,CA,San Bernardino County,6071,30243,633.0,12.0,1106.0,126600.0,12.0,14378.0,140990.0,1751.0,91065
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,KY,McCreary County,21147,2021,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,21147
1075,MT,Big Horn County,30003,2018,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,30003
1698,SD,Yankton County,46135,2021,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,46135
50,AL,Pickens County,1107,2021,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1107


### 4. Population Data

### POPULATION DATA FROM:
* https://repository.duke.edu/catalog/27251037-2a3b-462a-971d-4b023121bf7b (1990 - 2000)
* https://repository.duke.edu/catalog/f49b199b-1496-4636-91f3-36226c8e7f80 (2000 - 2010)
* https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates/2020-evaluation-estimates/2010s-counties-total.html (2010 - 2020, dropped 2020)
* https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html#v2023 (2020 - 2023)

In [38]:
#Loading in Duke Population Data (from Census)
duke_cols = ["fips","year","tot_pop"]
df_pop_1990 = pd.read_csv("Raw_Data/POP1990.csv",low_memory=False)
df_pop_2000 = pd.read_csv("Raw_Data/POP2000.csv",low_memory=False)
df_pop_duke = pd.concat([df_pop_1990,df_pop_2000],axis=0)
df_pop_duke = df_pop_duke[duke_cols]

#Loading in 2011-2019 Census Population Data
df_pop_2010 = pd.read_csv("Raw_Data/co-est2020-alldata.csv",low_memory=False)
pop_2010_cols = ['SUMLEV','REGION','DIVISION','STATE','COUNTY','STNAME','CTYNAME','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013', \
                 'POPESTIMATE2014','POPESTIMATE2015','POPESTIMATE2016','POPESTIMATE2017','POPESTIMATE2018','POPESTIMATE2019'] #dropping 2010 and 2019 (in other datasets)
df_pop_2010 = df_pop_2010[pop_2010_cols]

#Loading in 2020-2023 Census Population Data
df_pop_2020 = pd.read_csv("Raw_Data/co-est2023-alldata.csv",low_memory=False)
pop_2020_cols = ['SUMLEV','REGION',	'DIVISION','STATE','COUNTY','STNAME','CTYNAME','POPESTIMATE2020','POPESTIMATE2021','POPESTIMATE2022','POPESTIMATE2023']
df_pop_2020 = df_pop_2020[pop_2020_cols]

#Merging Census Population Data (2011 - 2023)
df_pop_census = df_pop_2010.merge(df_pop_2020, on = ["STATE","COUNTY","STNAME","SUMLEV","REGION","DIVISION","CTYNAME"])

# Renaming the Year columns
pop_cols = ['POPESTIMATE2011', 'POPESTIMATE2012','POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', \
            'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018', \
            'POPESTIMATE2019', 'POPESTIMATE2020', 'POPESTIMATE2021', \
            'POPESTIMATE2022', 'POPESTIMATE2023']
stripped_cols = [int(x[-4:]) for x in pop_cols]
df_pop_census['COUNTY'] = df_pop_census['COUNTY'].astype(str)
df_pop_census['COUNTY'] = df_pop_census['COUNTY'].str.zfill(3)
df_pop_census["fips"] = (df_pop_census["STATE"].astype(str) + df_pop_census["COUNTY"]).astype(int)
col_mapping = dict(zip(pop_cols, stripped_cols))
df_pop_census.rename(columns=col_mapping, inplace=True)

# Drop Unnecessary Columns
df_pop_census = df_pop_census[["fips"]+stripped_cols]

# Reshape the DataFrame to have Years in one Column and Total Population in other
df_pop_census = pd.melt(df_pop_census, id_vars=['fips'], var_name='year', value_name='tot_pop')

# Remove States from Dataset (fipps % 1000 = 0)
df_pop_census['year'] = df_pop_census['year'].astype(int)
df_pop_census = df_pop_census[df_pop_census["fips"]%1000 != 0] # drops state values

#Merge Population data (Duke + Census)
df_pop = pd.concat([df_pop_duke,df_pop_census],axis = 0)
df_pop.drop_duplicates(inplace=True)
df_pop.reset_index(drop=True, inplace=True)
df_pop["fips"] = df_pop["fips"].astype(int)
df_pop["tot_pop"] = df_pop["tot_pop"].astype(int)

# Report Fips with Missing Population Data
fips_counts = df_pop['fips'].value_counts()
incorrect_fips = fips_counts[fips_counts != 34].index

# Check if there are any incorrect FIPS codes
if incorrect_fips.empty:
    print("All FIPS codes have exactly 34 entries.")
else:
    print("The following FIPS codes do not have exactly 34 entries:")
    print(incorrect_fips.to_list())
    print("\nMissing years for each FIPS code:")

    # Loop through each FIPS code with incorrect counts
    for fip in incorrect_fips:
        # Filter population data for the current FIPS code
        fip_data = df_pop[df_pop['fips'] == fip]
        
        # Get the years present for the current FIPS code
        present_years = set(fip_data['year'])
        
        # Generate a list of missing years between 1990 and 2023
        missing_years = [year for year in range(1990, 2024) if year not in present_years]
        
        # Print missing years for the current FIPS code
        if missing_years:
            print(f"FIPS {fip} missing population data for years: {missing_years}")
        else:
            print(f"FIPS {fip} has data for all years between 1990 and 2023.")
df_pop_2024 = df_pop[df_pop["year"]==2023]   
df_pop_2024.loc[:,"year"] = [2024]*len(df_pop_2024)
df_pop = pd.concat([df_pop,df_pop_2024],axis=0)      
df_pop.sort_values(by=["fips","year"],inplace=True)
df_pop.reset_index(drop=True, inplace=True)
df_pop

The following FIPS codes do not have exactly 34 entries:
[8014, 2105, 2195, 2198, 2230, 2275, 9009, 46113, 9013, 51515, 9001, 9003, 9005, 9011, 9007, 9015, 2270, 2261, 2063, 2066, 2158, 46102, 2201, 2280, 2232, 51560]

Missing years for each FIPS code:
FIPS 8014 missing population data for years: [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
FIPS 2105 missing population data for years: [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
FIPS 2195 missing population data for years: [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
FIPS 2198 missing population data for years: [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
FIPS 2230 missing population data for years: [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
FIPS 2275 missing population data for years: [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
FIPS 9009 missing population data for years: [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,

,fips,year,tot_pop
0,1001,1990,34353
1,1001,1991,35010
2,1001,1992,35985
3,1001,1993,36953
4,1001,1994,38186
...,...,...,...
109868,56045,2020,6816
109869,56045,2021,6746
109870,56045,2022,6858
109871,56045,2023,6808


### 5. EV Car Registrations and Stations Plot

#### EV Registrations

In [39]:
ev_reg_data = pd.read_csv("Raw_Data/EV_registration_raw.csv")
ev_reg_data = ev_reg_data[["Year","State","Electric (EV)","Plug-In Hybrid Electric (PHEV)"]]
ev_reg_data["Total EV registration"] = ev_reg_data["Electric (EV)"] + ev_reg_data["Plug-In Hybrid Electric (PHEV)"]
ev_reg_data = ev_reg_data[["Year","State","Total EV registration"]]
ev_reg_data

,Year,State,Total EV registration
0,2016,Alabama,1400
1,2016,Alaska,400
2,2016,Arizona,9100
3,2016,Arkansas,700
4,2016,California,258200
...,...,...,...
352,2022,Virginia,78300
353,2022,Washington,135500
354,2022,West Virginia,3300
355,2022,Wisconsin,25700


#### Cummulative EV Stations

In [40]:
#Create Stations Copy
df_stations_cum = df_stations.copy()

#Converting State Abreviations to State Names
df_stations_cum['State'] = df_stations_cum['state'].apply(lambda x: abbreviation_to_state.get(x, x))
df_stations_cum.drop(columns=['state','county_id','id','county'],inplace=True)
df_stations_cum = df_stations_cum.groupby(by=["Year Opened","State"],as_index = False).sum()
df_stations_cum.sort_values(by=['State', 'Year Opened'], inplace=True)

# Get unique states and unique years
states = df_stations_cum['State'].unique()
min_year = min(df_stations_cum["Year Opened"])
all_years = range(min_year,2024)

# Create a DataFrame with all combinations of states and years
all_combinations = pd.MultiIndex.from_product([states, all_years], names=['State', 'Year Opened'])
missing_years = pd.DataFrame(index=all_combinations).reset_index()
df_stations_cum = pd.merge(missing_years, df_stations_cum, on=['State', 'Year Opened'], how='left')
df_stations_cum['Number_of_Stations'].fillna(0, inplace=True)
df_stations_cum['Number_of_Stations'] = df_stations_cum['Number_of_Stations'].astype(int)
df_stations_cum[df_stations_cum["State"] == 'Alaska']

# Calculate cumulative sum of stations for each state
df_stations_cum['Available Stations'] = df_stations_cum.groupby('State',as_index=False)['Number_of_Stations'].cumsum()
df_stations_cum['Year'] = df_stations_cum["Year Opened"]

# Drop the original 'number_of_stations' column
df_stations_cum.drop(columns=['Year Opened'], inplace=True)
df_stations_cum.rename(columns={'Number_of_Stations': 'Newly Opened Stations'}, inplace=True)

# Sort the DataFrame by 'State' and 'Year' for better visualization
df_stations_cum.sort_values(by=['State', 'Year'], inplace=True)
df_stations_cum = df_stations_cum[(df_stations_cum["Year"] >= 2016) & (df_stations_cum["Year"] <= 2022)]
df_stations_cum = df_stations_cum[["Year","State","Newly Opened Stations","Available Stations"]]
df_stations_cum

,Year,State,Newly Opened Stations,Available Stations
21,2016,Alabama,10,89
22,2017,Alabama,34,123
23,2018,Alabama,16,139
24,2019,Alabama,18,157
25,2020,Alabama,32,189
...,...,...,...,...
1473,2018,Wyoming,10,49
1474,2019,Wyoming,5,54
1475,2020,Wyoming,2,56
1476,2021,Wyoming,9,65


### 6. Saving DataFrames to Jsons/Csv

#### Changing States from Abreviations to Full Names

In [41]:
df_stations['state'] = df_stations['state'].apply(lambda x: abbreviation_to_state.get(x, x))
df_locations['state'] = df_locations['state'].apply(lambda x: abbreviation_to_state.get(x, x))
state_counties_all['state'] = state_counties_all['state'].apply(lambda x: abbreviation_to_state.get(x, x))
df_ports['state'] = df_ports['state'].apply(lambda x: abbreviation_to_state.get(x, x))
state_ids = pd.read_csv("/Users/laurenfrank/history-of-electric-vehicle-infrastructure-in-the-united-states/Preprocessing/Observable_Imports/state_ids.csv")
state_ids["state"] = state_ids['state'].apply(lambda x: abbreviation_to_state.get(x, x))

/var/folders/6l/z6x890s133g488rww9gwhhrc0000gn/T/ipykernel_77997/729157734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locations['state'] = df_locations['state'].apply(lambda x: abbreviation_to_state.get(x, x))


In [43]:
result_counties.to_json()
result_counties.to_file("counties_edited.json", driver="GeoJSON")
df_stations.to_csv("df_stations.csv",index = False)
df_locations.to_csv("df_locations.csv",index=False)
state_counties_all.to_csv("state_counties_all.csv",index=False)
df_ports.to_csv("df_ports.csv",index = False)
df_pop.to_csv("population.csv",index = False)
ev_reg_data.to_csv('ev_reg_data.csv',index=False)
df_stations_cum.to_csv("df_stations_cum.csv",index=False)
state_ids.to_csv("state_ids.csv",index=False)